Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [203]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [204]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [205]:
dist_matrix = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# i do dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] because combinations return only distinct pairs

In [206]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

Greedy means that we always choose the best option at each step, don't look far away in the future. 
*I am in a city and I go to the nearest city.*

In [207]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))

logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Abakan -> Krasnoyarsk (276.58km)
DEBUG:root:step: Krasnoyarsk -> Achinsk (161.71km)
DEBUG:root:step: Achinsk -> Kemerovo (296.59km)
DEBUG:root:step: Kemerovo -> Leninsk‐Kuznetskiy (74.76km)
DEBUG:root:step: Leninsk‐Kuznetskiy -> Prokopyevsk (91.87km)
DEBUG:root:step: Prokopyevsk -> Novokuznetsk (30.63km)
DEBUG:root:step: Novokuznetsk -> Biysk (187.38km)
DEBUG:root:step: Biysk -> Barnaul (132.82km)
DEBUG:root:step: Barnaul -> Novosibirsk (194.50km)
DEBUG:root:step: Novosibirsk -> Tomsk (206.90km)
DEBUG:root:step: Tomsk -> Seversk (14.97km)
DEBUG:root:step: Seversk -> Rubtsovsk (613.13km)
DEBUG:root:step: Rubtsovsk -> Omsk (647.47km)
DEBUG:root:step: Omsk -> Tobolsk (475.40km)
DEBUG:root:step: Tobolsk -> Tyumen (200.98km)
DEBUG:root:step: Tyumen -> Kurgan (189.69km)
DEBUG:root:step: Kurgan -> Kopeysk (236.87km)
DEBUG:root:step: Kopeysk -> Chelyabinsk (14.72km)
DEBUG:root:step: Chelyabinsk -> Miass (87.20km)
DEBUG:root:step: Miass -> Zlatoust (33.88km)
DEBUG:root:step: Zl

## Second Greedy Algorithm

in this case, we look for a "segment". This algorithm lead to build a tree without cycles using segments.

In [208]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

`segments` : each element of the list is a tuple of two cities and the distance between them. So `segments` contains all the possible segments between the cities.

In [209]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]
visited = set()
edges = set()

Order the segments by distance and then we start from the shortest one. If the two cities of the segment are not already in the tree, we add the segment to the tree.

In [210]:
shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))
visited |= shortest[0]
visited
edges |= {tuple(shortest[0])}
segments = [s for s in segments if not cyclic(edges | {tuple(s[0])})]

In [211]:
segments

[({0, 1}, 291.12871812096597),
 ({0, 2}, 2516.33442321499),
 ({0, 3}, 843.679166213213),
 ({0, 4}, 3050.3327570180104),
 ({0, 5}, 3697.257751542974),
 ({0, 6}, 3169.5429738640505),
 ({0, 7}, 3025.825566168284),
 ({0, 8}, 3160.2480682832283),
 ({0, 9}, 2898.5097152436815),
 ({0, 10}, 3364.293337838562),
 ({0, 11}, 510.48040419758183),
 ({0, 12}, 3659.116686030305),
 ({0, 13}, 3674.232708952557),
 ({0, 14}, 2194.6122160833356),
 ({0, 15}, 439.31947457577616),
 ({0, 16}, 2481.4335921593824),
 ({0, 17}, 716.7680282426004),
 ({0, 18}, 3680.756755100788),
 ({0, 19}, 2796.2681940600255),
 ({0, 20}, 1938.0889218280688),
 ({0, 21}, 3264.6854322830545),
 ({0, 22}, 3678.9247499298162),
 ({0, 23}, 1488.4462769418906),
 ({0, 24}, 3412.8183943767885),
 ({0, 25}, 2707.300979312825),
 ({0, 26}, 3021.42923733918),
 ({0, 27}, 3335.5365148607657),
 ({0, 28}, 3414.741242958905),
 ({0, 29}, 3026.843158723314),
 ({0, 30}, 3501.175002654965),
 ({0, 31}, 872.0827902020271),
 ({0, 32}, 3145.496905689025),
 ({0

In [212]:
cyclic(edges)

False

In [213]:
import random

def simulated_annealing_tsp(dist_matrix, initial_tsp, initial_temp, cooling_rate, num_iterations):
    def scramble_mutation(tsp):
        a, b = sorted(random.sample(range(1, len(tsp) - 1), 2))
        tsp[a:b] = random.sample(tsp[a:b], len(tsp[a:b]))
        return tsp

    def tsp_cost(tsp):
        return sum(dist_matrix[tsp[i], tsp[i + 1]] for i in range(len(tsp) - 1))

    current_tsp = initial_tsp[:]
    current_cost = tsp_cost(current_tsp)
    best_tsp = current_tsp[:]
    best_cost = current_cost
    temp = initial_temp

    for i in range(num_iterations):
        new_tsp = scramble_mutation(current_tsp[:])
        new_cost = tsp_cost(new_tsp)
        if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temp):
            current_tsp = new_tsp
            current_cost = new_cost
            if new_cost < best_cost:
                best_tsp = new_tsp
                best_cost = new_cost
        temp *= cooling_rate

    return best_tsp, best_cost

# Parametri per Simulated Annealing
initial_temp = 1000
cooling_rate = 0.995
num_iterations = 10000

# TSP iniziale (usiamo il percorso trovato dal nearest neighbor come punto di partenza)
initial_tsp = tsp_path

# Esegui Simulated Annealing
optimized_tsp, optimized_cost = simulated_annealing_tsp(DIST_MATRIX, initial_tsp, initial_temp, cooling_rate, num_iterations)

logging.info(f"result: Found an optimized path of {len(optimized_tsp)-1} steps, total length {optimized_cost:.2f}km")
print(f"Optimized TSP Path: {optimized_tsp}")
print(f"Optimized Total Cost: {optimized_cost:.2f} km")

INFO:root:result: Found an optimized path of 46 steps, total length 44628.65km


Optimized TSP Path: [0, np.int64(33), np.int64(25), np.int64(19), np.int64(44), np.int64(4), np.int64(32), np.int64(43), np.int64(18), np.int64(12), np.int64(28), np.int64(5), np.int64(22), np.int64(45), np.int64(30), np.int64(42), np.int64(9), np.int64(20), np.int64(40), np.int64(3), np.int64(23), np.int64(6), np.int64(41), np.int64(14), np.int64(39), np.int64(10), np.int64(35), np.int64(34), np.int64(13), np.int64(38), np.int64(37), np.int64(7), np.int64(26), np.int64(21), np.int64(2), np.int64(17), np.int64(16), np.int64(31), np.int64(1), np.int64(15), np.int64(11), np.int64(36), np.int64(24), np.int64(8), np.int64(29), np.int64(27), 0]
Optimized Total Cost: 44628.65 km


## Genetic Algorithm
- steady state: offsprings compete with the parents
- parent selection: we try different methods
- mutation: we try different methods (scramble, inversion)
- crossover: we try different methods (edge recombination, inver over)

Fitness is defined as the inverse of the sum of the distance between adjacent cities and the distance between the first and the last city.

In [214]:
def fitness(path):
    distance = 0
    for i in range(len(path) - 1):
        distance += DIST_MATRIX[path[i], path[i + 1]]
    distance += DIST_MATRIX[path[-1], path[0]]
    return 1/distance  # greater is the distance lower (worse) is the fitness

### Parent Selection

In [215]:
def parent_selection(population, fitnesses, method, num_parents = 20):
    parents = []
    if method == 0 : # Roulette Wheel
        total_fitness = sum(fitnesses)
        relative_fitness = [f/total_fitness for f in fitnesses]  # 
        # Select num_parents from the population using relative fitness
        mating_pool = np.random.choice(range(len(population)), num_parents, p=relative_fitness)
        parents = [population[i] for i in mating_pool]
        
    elif method == 1: # Tournament Selection
        for _ in range(num_parents):
            # Select random indices for the tournament
            tournament_indices = np.random.choice(range(len(population)), int(0.3 * len(population)), replace=False)
            tournament_fitnesses = [fitnesses[i] for i in tournament_indices]
            
            # Select the individual with the best fitness in the tournament
            best_index = tournament_indices[np.argmax(tournament_fitnesses)]
            parents.append(population[best_index])
    
    elif method == 2: # Rank Selection
        rank = np.argsort(fitnesses)
        for i in range(num_parents):
            parents.append(population[rank[i]])
            
    else:
        logging.error("Invalid parent selection method")

    return parents

### Crossover

In [216]:
# edge recombination operator crossover

def edge_recombination(parent1, parent2):
    # Step 1: Create an adjacency list
    adjacency_list = {}
    for city in parent1:
        adjacency_list[city] = set()

    # Populate adjacency lists with neighbors from parent1
    for i in range(len(parent1)):
        left_neighbor = parent1[i - 1] if i > 0 else parent1[-1]
        right_neighbor = parent1[i + 1] if i < len(parent1) - 1 else parent1[0]
        adjacency_list[parent1[i]].update([left_neighbor, right_neighbor])

    # Populate adjacency lists with neighbors from parent2
    for i in range(len(parent2)):
        left_neighbor = parent2[i - 1] if i > 0 else parent2[-1]
        right_neighbor = parent2[i + 1] if i < len(parent2) - 1 else parent2[0]
        adjacency_list[parent2[i]].update([left_neighbor, right_neighbor])

    # Step 2: Initialize the offspring with the first city from parent1
    current_city = parent1[0]
    offspring = [current_city]

    # Step 3: Generate the rest of the offspring sequence
    while len(offspring) < len(parent1):
        # Remove the current city from all neighbors in the adjacency list
        for neighbors in adjacency_list.values():
            neighbors.discard(current_city)

        # Find the next city with the fewest neighbors
        if adjacency_list[current_city]:
            next_city = min(adjacency_list[current_city], key=lambda x: len(adjacency_list[x]))
        else:
            # If no adjacent city is available, pick a random city not in offspring
            next_city = random.choice([city for city in parent1 if city not in offspring])

        offspring.append(next_city)
        current_city = next_city

    return offspring

def inver_over(parent1, parent2, p=0.5):
    # Initialize offspring as a copy of parent1
    offspring = parent1[:]
    n = len(offspring)
    
    # Set of cities already processed
    visited = set()

    # Start from a random city in parent1
    current_city = random.choice(offspring)
    visited.add(current_city)

    while len(visited) < n:
        # With probability p, pick the current city from the other parent
        if random.random() < p:
            # Choose the next city based on parent2's adjacency
            next_city_index = parent2.index(current_city)
            if next_city_index == n - 1:
                next_city = parent2[0]  # Wrap around
            else:
                next_city = parent2[next_city_index + 1]
        else:
            # Choose a random unvisited city
            next_city = random.choice([city for city in offspring if city not in visited])

        # Find positions of current and next city in offspring
        current_index = offspring.index(current_city)
        next_index = offspring.index(next_city)

        # Invert the segment between current and next city
        if current_index < next_index:
            offspring[current_index:next_index + 1] = reversed(offspring[current_index:next_index + 1])
        else:
            # Wrap around if next_index is "before" current_index
            segment = offspring[current_index:] + offspring[:next_index + 1]
            segment.reverse()
            offspring[current_index:] = segment[:n - current_index]
            offspring[:next_index + 1] = segment[n - current_index:]

        # Mark the next city as visited and move to it
        visited.add(next_city)
        current_city = next_city

    return offspring

### Mutation

In [224]:
def scramble_mutation(path):
    # Choose two random points in the sequence to define the subset
    start, end = sorted(random.sample(range(len(path)), 2))

    # Scramble the subset
    subset = path[start:end + 1]
    random.shuffle(subset)

    # Place the scrambled subset back in the path
    mutated_path = path[:start] + subset + path[end + 1:]
    return mutated_path


def inversion_mutation(path):
    # Choose two random points in the path
    start, end = sorted(random.sample(range(len(path)), 2))

    # Reverse the subset
    mutated_path = path[:start] + path[start:end + 1][::-1] + path[end + 1:]
    return mutated_path


def adaptive_mutation(path, generation, max_generations, mutation_fn):
    """Mutation rate adapts based on generation progress"""
    # Higher mutation rate early, lower later
    mutation_rate = 0.3 * (1 - generation / max_generations)
    
    if random.random() < mutation_rate:
        return mutation_fn(path)
    return path

### Genetic Algorithm

In [225]:
def print_path_details(path, cities_df, dist_matrix):
    """
    Print detailed information about a TSP path including city-to-city steps and distances
    
    Parameters:
    path: list of indices representing the city order
    cities_df: DataFrame containing city information with 'name' column
    dist_matrix: distance matrix between cities
    """
    total_distance = 0
    steps = 0
    
    # Convert numpy integers to regular Python integers
    path = [int(i) for i in path]
    
    # Print each step in the path
    for i in range(len(path)):
        current_city = cities_df.iloc[path[i]]['name']
        next_city = cities_df.iloc[path[(i + 1) % len(path)]]['name']
        distance = dist_matrix[path[i], path[(i + 1) % len(path)]]
        
        logging.debug(f"step: {current_city} -> {next_city} ({distance:.2f}km)")
        total_distance += distance
        steps += 1
    
    logging.info(f"result: Found a path of {steps} steps, total length {total_distance:.2f}km")
    return total_distance, steps


In [228]:
import numpy as np

def genetic_algorithm(population_size, num_generations, crossover_fn, mutation_fn, parent_selection_fn, p_selection_method):
    # Set replacement rate - 30% of population
    replacement_rate = 0.5
    num_replacements = int(population_size * replacement_rate)
    
    # Initialize the population with random TSP paths
    population = [list(np.random.permutation(len(CITIES))) for _ in range(population_size)]
    best_path = None
    best_fitness = float('-inf')

    # Evaluate the initial population
    fitness_values = [fitness(path) for path in population]

    for generation in range(num_generations):
        # Select a few parents for crossover
        parents = parent_selection_fn(population, fitness_values, p_selection_method)

        # Generate offspring by crossover and mutation
        next_generation = []
        for i in range(0, len(parents), 2):
            child1 = crossover_fn(parents[i], parents[i + 1])
            child2 = crossover_fn(parents[i + 1], parents[i])
            next_generation.extend([adaptive_mutation(child1,generation,num_generations,mutation_fn), adaptive_mutation(child2,generation,num_generations,mutation_fn)])

        # Calculate fitness for the offspring
        offspring_fitness = [fitness(child) for child in next_generation]

        # Replace the least fit individuals with new offspring
        for child, child_fitness in zip(next_generation[:num_replacements], offspring_fitness[:num_replacements]):
            # Find the index of the least fit individual in the population
            least_fit_index = np.argmin(fitness_values)
            if child_fitness > fitness_values[least_fit_index]:
                # Replace least fit individual with the new child
                population[least_fit_index] = child
                fitness_values[least_fit_index] = child_fitness

                # Update best path and fitness if needed
                if child_fitness > best_fitness:
                    best_path = child
                    best_fitness = child_fitness

        if generation % 10 == 0:
            logging.info(f"Generation {generation}: Best Fitness = {1/best_fitness:.4f}")

    
            
    # Print final path details
    logging.info("Final path details:")
    total_distance, steps = print_path_details(best_path, CITIES, DIST_MATRIX)
    
    # Print the actual path of city names for easier comparison
    city_path = [CITIES.iloc[i]['name'] for i in best_path]
    logging.info(f"Optimized TSP Path: {city_path}")
    logging.info(f"Optimized Total Cost: {1/best_fitness:.2f} km")
    
    
    return best_path, best_fitness


## Iterations

In [227]:
# Params for Genetic Algorithm
population_size = 200
num_generations = 10000
crossover_fn = edge_recombination
mutation_fn = inversion_mutation
parent_selection_fn = parent_selection
p_selection_method = 0  # 0: Roulette Wheel, 1: Tournament Selection, 2: Rank Selection

# Run Genetic Algorithm
best_path, best_fitness = genetic_algorithm(population_size, num_generations, crossover_fn, mutation_fn, parent_selection_fn, p_selection_method)



INFO:root:Generation 0: Best Fitness = 322984.3257
INFO:root:Generation 10: Best Fitness = 304677.6960
INFO:root:Generation 20: Best Fitness = 283430.5407
INFO:root:Generation 30: Best Fitness = 268035.8847
INFO:root:Generation 40: Best Fitness = 258764.9824
INFO:root:Generation 50: Best Fitness = 255146.7665
INFO:root:Generation 60: Best Fitness = 243123.8839
INFO:root:Generation 70: Best Fitness = 241981.3710
INFO:root:Generation 80: Best Fitness = 234134.9339
INFO:root:Generation 90: Best Fitness = 213887.3223
INFO:root:Generation 100: Best Fitness = 213887.3223
INFO:root:Generation 110: Best Fitness = 206749.3541
INFO:root:Generation 120: Best Fitness = 197776.2914
INFO:root:Generation 130: Best Fitness = 197776.2914
INFO:root:Generation 140: Best Fitness = 194744.9715
INFO:root:Generation 150: Best Fitness = 194744.9715
INFO:root:Generation 160: Best Fitness = 192474.1300
INFO:root:Generation 170: Best Fitness = 179888.8513
INFO:root:Generation 180: Best Fitness = 176380.4450
INFO

Optimized TSP Path: [np.int32(135), np.int32(120), np.int32(160), np.int32(33), np.int32(49), np.int32(96), np.int32(32), np.int32(25), np.int32(105), np.int32(137), np.int32(97), np.int32(40), np.int32(0), np.int32(17), np.int32(1), np.int32(65), np.int32(88), np.int32(153), np.int32(73), np.int32(112), np.int32(136), np.int32(145), np.int32(121), np.int32(89), np.int32(128), np.int32(64), np.int32(8), np.int32(72), np.int32(152), np.int32(56), np.int32(129), np.int32(80), np.int32(104), np.int32(58), np.int32(57), np.int32(90), np.int32(113), np.int32(11), np.int32(54), np.int32(109), np.int32(86), np.int32(15), np.int32(75), np.int32(82), np.int32(3), np.int32(31), np.int32(23), np.int32(151), np.int32(71), np.int32(146), np.int32(6), np.int32(61), np.int32(16), np.int32(48), np.int32(162), np.int32(106), np.int32(41), np.int32(144), np.int32(157), np.int32(139), np.int32(124), np.int32(93), np.int32(78), np.int32(92), np.int32(142), np.int32(132), np.int32(20), np.int32(91), np.int